In [9]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [10]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        demoPaths.append(os.path.join(dir_path, filename))
    return demoPaths

In [11]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [12]:
def generateData(parsers):
    killData, roundData, damageData = [], [], [],
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (killData, roundData, damageData)

In [13]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [14]:
def createSidedCSVDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []

    Kills = []
    Deaths = []
    KdDifferential = []
    DamageDealt = []
    DamageTaken = []
    DamageDifferential = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            
            totalRounds.append(tempTotalRounds)
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [15]:
demo_path = '/mnt/d/Top_Players/ESL_Pro_League_Season_13'

In [16]:
output_path = './damageDifferentials/epls13/eslInfernoDifferentials.csv'

In [17]:
demos = getDemoPaths(demo_path)

In [18]:
infernoDemos = []
for x in demos:
    if "inferno" in x:
        infernoDemos.append(x)

In [19]:
infernoMappingPath = './mappings/mappings/de_inferno_place_mappings.txt'

In [20]:
file = open(infernoMappingPath, 'r')
lines = file.readlines()
file.close()

In [21]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [22]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
            mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [23]:
parsers = generateDemoParsers(infernoDemos)


14:01:27 [INFO] Go version>=1.14.0
14:01:27 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/ESL_Pro_League_Season_13/astralis-vs-endpoint-m1-inferno.dem
14:01:27 [INFO] Setting demo id to astralis-vs-endpoint-m1-inferno
14:01:27 [INFO] Setting parse rate to 128
14:01:28 [INFO] Go version>=1.14.0
14:01:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/ESL_Pro_League_Season_13/astralis-vs-evil-geniuses-m3-inferno.dem
14:01:28 [INFO] Setting demo id to astralis-vs-evil-geniuses-m3-inferno
14:01:28 [INFO] Setting parse rate to 128
14:01:28 [INFO] Go version>=1.14.0
14:01:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/ESL_Pro_League_Season_13/astralis-vs-virtus-pro-m1-inferno.dem
14:01:28 [INFO] Setting demo id to astralis-vs-virtus-pro-m1-inferno
14:01:28 [INFO] Setting parse rate to 128
14:01:28 [INFO] Go version>=1.14.0
14:01:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/ESL_Pro_League_Season_13/complex

# Demo Parsers: 29


In [24]:
infernoKills, infernoROunds, infernoDamages = generateData(parsers)


14:01:30 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:01:30 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/astralis-vs-endpoint-m1-inferno.dem
14:01:50 [INFO] Wrote demo parse output to astralis-vs-endpoint-m1-inferno.json
14:01:50 [INFO] Reading in JSON from astralis-vs-endpoint-m1-inferno.json
14:01:50 [INFO] JSON data loaded, available in the `json` attribute to parser
14:01:50 [INFO] Successfully parsed JSON output
14:01:50 [INFO] Successfully returned JSON output
14:01:50 [INFO] Parsed kills to Pandas DataFrame
14:01:50 [INFO] Parsed rounds to Pandas DataFrame
14:01:50 [INFO] Parsed damages to Pandas DataFrame
14:01:50 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:01:50 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/astralis-vs-evil-geniuses-m3-inferno.dem
14:02:06 [INFO] Wrote demo parse output to astralis-vs-evil-geniuses-m3-inferno.json
14:02:06 [INFO] Reading in JSON 

14:04:45 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/furia-vs-gambit-m2-inferno.dem
14:05:03 [INFO] Wrote demo parse output to furia-vs-gambit-m2-inferno.json
14:05:03 [INFO] Reading in JSON from furia-vs-gambit-m2-inferno.json
14:05:04 [INFO] JSON data loaded, available in the `json` attribute to parser
14:05:04 [INFO] Successfully parsed JSON output
14:05:04 [INFO] Successfully returned JSON output
14:05:04 [INFO] Parsed kills to Pandas DataFrame
14:05:04 [INFO] Parsed rounds to Pandas DataFrame
14:05:04 [INFO] Parsed damages to Pandas DataFrame
14:05:04 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:05:04 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/g2-vs-faze-m2-inferno.dem
14:05:20 [INFO] Wrote demo parse output to g2-vs-faze-m2-inferno.json
14:05:20 [INFO] Reading in JSON from g2-vs-faze-m2-inferno.json
14:05:21 [INFO] JSON data loaded, available in the `json` attribute to parser
14:05:21 [INFO] Su

14:08:24 [INFO] JSON data loaded, available in the `json` attribute to parser
14:08:24 [INFO] Successfully parsed JSON output
14:08:24 [INFO] Successfully returned JSON output
14:08:24 [INFO] Parsed kills to Pandas DataFrame
14:08:24 [INFO] Parsed rounds to Pandas DataFrame
14:08:24 [INFO] Parsed damages to Pandas DataFrame
14:08:24 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:08:24 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/virtus-pro-vs-ence-m1-inferno.dem
14:08:38 [INFO] Wrote demo parse output to virtus-pro-vs-ence-m1-inferno.json
14:08:38 [INFO] Reading in JSON from virtus-pro-vs-ence-m1-inferno.json
14:08:39 [INFO] JSON data loaded, available in the `json` attribute to parser
14:08:39 [INFO] Successfully parsed JSON output
14:08:39 [INFO] Successfully returned JSON output
14:08:39 [INFO] Parsed kills to Pandas DataFrame
14:08:39 [INFO] Parsed rounds to Pandas DataFrame
14:08:39 [INFO] Parsed damages to Pandas DataFrame
1

In [25]:
for x in range(len(infernoKills)):
    if(infernoKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [26]:
infernoKill = concat_data(infernoKills)
infernoRound = concat_data(infernoROunds)
infernoDamage = concat_data(infernoDamages)

In [27]:
for x in range(len(infernoKill)):
    if  infernoKill.iloc[x]['AttackerAreaName'] != None:
        if infernoKill.iloc[x]['AttackerAreaName'] in mappedPlaceNames:
            infernoKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[infernoKill.iloc[x]['AttackerAreaName']]
        if infernoKill.iloc[x]['VictimAreaName'] in mappedPlaceNames:
            infernoKill.at[x, 'VictimAreaName'] = mappedPlaceNames[infernoKill.iloc[x]['VictimAreaName']]

for x in range(len(infernoDamage)):
    if  infernoDamage.iloc[x]['AttackerAreaName'] != None:
        if infernoDamage.iloc[x]['AttackerAreaName'] in mappedPlaceNames:
            infernoDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[infernoDamage.iloc[x]['AttackerAreaName']]
        if infernoDamage.iloc[x]['VictimAreaName'] in mappedPlaceNames:
            infernoDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[infernoDamage.iloc[x]['VictimAreaName']]

In [28]:
player_names = list(infernoKill.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [29]:
infernoDifferentials = createSidedCSVDF(player_name, infernoDamage, infernoKill, infernoRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [30]:
infernoDifferentials[infernoDifferentials['Location'] == 'BombsiteB']

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds
3856,BombsiteB,MiGHTYMAX,0,0,0,0,0,0,CT,133
3857,BombsiteB,MiGHTYMAX,0,1,-1,0,8,-8,T,133
5356,BombsiteB,JACKZ,0,0,0,0,0,0,CT,59
5357,BombsiteB,JACKZ,0,1,-1,0,4,-4,T,59


In [31]:
mappedPlaceNames

{'AbovePit': 'Deep T Spawn',
 'Admin': 'T Ramp Ledge',
 'PalaceInterior': 'T Ramp Peeking Towards Mid',
 'Airplane': 'Bottom Mid exposed to Top Mid',
 'Alley': 'Bottom Mid Banana',
 'APlatform': 'Banana',
 'Arch': 'CT Arch',
 'Atrium': 'Logs',
 'Attic': 'Chokepoint Entering B',
 'Back': 'Grill',
 'BackAlley': 'CT Peek Towards B Choke',
 'BackAlleys': '1st Oranges',
 'BackCourtyard': '2nd Oranges',
 'BackDoor': 'Quad Box',
 'BackEntrance': 'Fountain',
 'BackHall': 'Dark',
 'BackofA': 'Coffins',
 'BackofB': 'Garden',
 'BackRoad': 'Construction',
 'BackRoom': 'Construction Sandbags',
 'BackWay': 'CT Truck',
 'BackYard': 'CT Tree',
 'Banana': 'CT Near Well',
 'BankExterior': 'CT Spawn ',
 'BankInterior': 'CT Speed Way',
 'Barn': 'CT Outside Arch',
 'BarnRoof': 'CT Outside Kitchen',
 'Basement': 'CT Arch',
 'Bathroom': 'Arch Side Cubby',
 'BDoors': 'Arch Side',
 'Beach': 'Library',
 'Bedroom': 'Top Mid Arch Side',
 'BigForest': 'Top Mid Lane Side ',
 'BigOffice': 'Top Mid',
 'BoatBar': 'Lan

In [32]:
infernoDifferentials

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds
0,Lane Side,Ax1Le,5,2,3,523,200,323,CT,160
1,Lane Side,Ax1Le,2,2,0,323,378,-55,T,160
2,Top Balcony,Ax1Le,4,4,0,647,481,166,CT,160
3,Top Balcony,Ax1Le,5,6,-1,458,532,-74,T,160
4,Top Mid Chokepoint,Ax1Le,2,1,1,336,176,160,CT,160
...,...,...,...,...,...,...,...,...,...,...
8029,Quad Box,pesadelo,0,1,-1,0,68,-68,T,21
8030,A Bombsite Close Left,pesadelo,0,0,0,0,0,0,CT,21
8031,A Bombsite Close Left,pesadelo,0,0,0,0,6,-6,T,21
8032,CT Peek Towards B Choke,pesadelo,0,0,0,0,0,0,CT,21


In [36]:
infernoDifferentials.to_csv(output_path)

In [37]:
print(output_path)

./damageDifferentials/epls13/eslInfernoDifferentials.csv
